## Credits

This notebook was developed based on example notebook provided by AWS Sagemaker -> https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/scikit_bring_your_own.
We focus on improving the training part and add GPU support for computations and Keras library

## Build Docker container

This container will be needed to provide all learning and GPU-required dependencies as well as to expose simple API so that Sagemaker knows how to use it.

We can see that Tensorflow has the following requirements (https://www.nvidia.com/en-us/data-center/gpu-accelerated-applications/tensorflow/):

* 64-bit Linux
* Python 2.7
* CUDA 7.5 (CUDA 8.0 required for Pascal GPUs)
* cuDNN v5.1 (cuDNN v6 if on TF v1.3)

To fullfill them we can use prebuilt image provided by nvidia-docker project (see https://github.com/NVIDIA/nvidia-docker). We do not need to worry about nvidia-docker deamon since it is provided by Sagemaker.

In [3]:
!cat container/Dockerfile

# Build an image that can do training  in SageMaker
# This image contains CUDA 9.0 (CUDA libs are backward-compatible), cuddn version 7 and 64bit ubuntu
FROM nvidia/cuda:9.0-cudnn7-runtime-ubuntu16.04

# FROM ubuntu:16.04 - if we do not want cuda support
MAINTAINER Amazon AI <sage-learner@amazon.com>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
         python-dev \
         python-tk \
         gcc \
         g++ \
         libopenblas-dev \
    && rm -rf /var/lib/apt/lists/*

# Here we get all python packages.
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py && \
    pip install numpy scikit-learn pandas flask gevent gunicorn matplotlib tensorflow-gpu keras Pillow six

# Set some environment variables. PYTHONUNBUFFERED keeps Python from buffering our standard
# output stream, which means that logs can be delivered to the user quickly. PYTHONDONTWRITEBYTECODE
# kee

In [4]:
%%sh

# AWS script to run the build and deployment 

# The name of our algorithm
algorithm_name=keras-nn

cd container

chmod +x keras-nn/train

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/9 : FROM nvidia/cuda:9.0-cudnn7-runtime-ubuntu16.04
 ---> 03608776d6cb
Step 2/9 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> c46e372d6308
Step 3/9 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates          python-dev          python-tk          gcc          g++          libopenblas-dev     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 332aac28bcee
Step 4/9 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py &&     pip install numpy scikit-learn pandas flask gevent gunicorn matplotlib tensorflow-gpu keras Pillow six
 ---> Using cache
 ---> d89a824d5f90
Step 5/9 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> 20be4641b434
Step 6/9 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 65db18abe65b
Step 7/9 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 6f9d18e96274
Step 8/9 : COPY keras-nn /opt/program
 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


In [5]:
# S3 prefix
prefix = 'DEMO-keras-nn'

import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [6]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [7]:
# Upload the train data. In this case we leave this directory empty, since the CIFAR data is builtin in Keras library.
# Otherwise we could access all the data we put on S3 from our container.
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/keras-nn:latest'.format(account, region)

tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.p2.xlarge',
                       output_path="s3://{}/output_keras_gpu".format(sess.default_bucket()),
                       sagemaker_session=sess)

tree.fit(data_location)

INFO:sagemaker:Creating training-job with name: keras-nn-2018-07-26-09-03-41-932


............................
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/usr/local/lib/python2.7/dist-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/usr/local/lib/python2.7/dist-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWar

170500096/170498071 [==============================] - 16s 0us/step

170508288/170498071 [==============================] - 16s 0us/step
Epoch 1 of 240

/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


476/500 [===========================>..] - ETA: 14s


499/500 [============================>.] - ETA: 0s
Testing for epoch 7:
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.45 | 1.18            | 0.27 
generator (test)       | 8.39 | 0.07            | 8.32 
discriminator (train)  | 1.39 | 0.57            | 0.82 
discriminator (test)   | 13.67 | 3.95            | 9.72 
Epoch 8 of 240

121/500 [======>.......................] - ETA: 3:49


...

195/500 [==========>...................] - ETA: 3:04


343/500 [===================>..........] - ETA: 1:35


491/500 [============================>.] - ETA: 5s


499/500 [============================>.] - ETA: 0s
Testing for epoch 9:
component              | loss | generation_loss | auxiliary_loss
-----------------------------------------------------------------
generator (train)      | 1.36 | 1.11            | 0.25 
generator (test)       | 9.50 | 0.01            | 9.49 
discriminator (train)  | 1.34 | 0.58            | 0.77 
discriminator (test)   | 14.82 | 4.68            | 10.15
Epoch 10 of 240

129/500 [======>.......................] - ETA: 3:45


228/500 [============>.................] - ETA: 2:44


In [29]:
!tar chvfz notebook2.tar.gz *

container/
container/ReadMe.md
container/keras-nn/
container/keras-nn/train_cpu
container/keras-nn/.ipynb_checkpoints/
container/keras-nn/train_gpu
container/keras-nn/train_old
container/keras-nn/Minibatch.py
container/keras-nn/train
container/Dockerfile
container/.ipynb_checkpoints/
container/Dockerfile_cpu
container/local_test/
container/local_test/test_dir/
container/local_test/test_dir/model/
container/local_test/test_dir/model/decision-tree-model.pkl
container/local_test/test_dir/output/
container/local_test/test_dir/output/success
container/local_test/test_dir/input/
container/local_test/test_dir/input/config/
container/local_test/test_dir/input/config/hyperparameters.json
container/local_test/test_dir/input/config/resourceConfig.json
container/local_test/test_dir/input/data/
container/local_test/test_dir/input/data/training/
container/local_test/test_dir/input/data/training/iris.csv
container/local_test/.ipynb_checkpoints/
data/
data/iris.csv
keras_on_gpu.ipynb
notebook.tar.gz
s